In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'designs')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
designs_palette = metadata_dict['color']
designs_markers = metadata_dict['markers']

Load data from 293T, MEF lenti infections (`data_lenti`)

In [ ]:
data_lenti, quantiles_lenti, stats_lenti, _ = base.load_data(base_path, metadata_path, 'lenti')

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
scatter_kwargs['jitter'] = 0.1
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['1.5-column'], 4.5))
fig_gridspec = matplotlib.gridspec.GridSpec(3, 6, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=([1.25,1.75,1.5]), width_ratios=[1,1.2,0.8,1.2,0.8,1])

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,:3]),
    'C': fig.add_subfigure(fig_gridspec[1,3:]),
    'D': fig.add_subfigure(fig_gridspec[2,:2]),
    'D2': fig.add_subfigure(fig_gridspec[2,2:4]),
    'D3': fig.add_subfigure(fig_gridspec[2,4:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    if '3' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_designs'/'fig_designs.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
def designs_stats_plots(axes, plot_df, marker_baseline=None, ylim_mean=None, ylim_slope=(0,1.2)):

    xlim = (-0.5, len(plot_df['design'].unique())-0.5)

    # stat gmean
    ax = axes[0]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                      ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Output mean', xlim=xlim, xlabel='design', ylabel='', yscale='log', ylim=ylim_mean,)
    if marker_baseline: ax.axhline(marker_baseline, color='black', ls=':')
    ax.minorticks_off()

    # slope
    ax = axes[1]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                      ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=ylim_slope,
        yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

    for ax in axes: 
        sns.despine(ax=ax)
        ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])

    # perform statistical tests
    pairs = [('base','CL'), ('OL','CL')]
    designs = plot_df['design'].unique()
    f, axes = plt.subplots(1,len(designs)-1, figsize=(5,2), gridspec_kw=dict(wspace=0.5))
    print('-------- OUTPUT MEAN --------\n')
    for ax, design in zip(axes, designs[1:]):
        print('Design: '+str(design))
        test_df = plot_df[(plot_df['design']==design) | (plot_df['group']=='base')]
        sns.stripplot(data=test_df, x='ts_label', y='output_gmean', ax=ax, hue='construct', palette=designs_palette, legend=False)
        ax.set(ylabel='', xlabel='', title=str(design), yscale='log')

        annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y='output_gmean')
        annotator.configure(**annotate_kwargs,).apply_and_annotate()
        print('\n')
    axes[0].set(ylabel='Output mean')

    f, axes = plt.subplots(1,len(designs)-1, figsize=(5,2), gridspec_kw=dict(wspace=0.5))
    print('-------- SLOPE --------\n')
    for ax, design in zip(axes, designs[1:]):
        print('Design: '+str(design))
        test_df = plot_df[(plot_df['design']==design) | (plot_df['group']=='base')]
        sns.stripplot(data=test_df, x='ts_label', y='slope', ax=ax, hue='construct', palette=designs_palette, legend=False)
        ax.set(ylabel='', xlabel='', title=str(design))

        annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y='slope')
        annotator.configure(**annotate_kwargs,).apply_and_annotate()
        print('\n')
    axes[0].set(ylabel='Slope')

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.65, right=0.12)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.45))

# transfection
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['ts_num']==1)) | (stats['group']=='base')) &
                       (stats['promoter']=='EF1a')]
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()

designs_stats_plots(axes, plot_df, marker_baseline=marker_baseline, ylim_mean=(2e1,2e4), ylim_slope=(0,1.3))

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.65, right=0.12)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.4))

# lenti (HEK293T)
plot_df = stats_lenti[stats_lenti['group'].isin(['controller','base']) & (stats_lenti['dox']==1000) & (stats_lenti['moi']==1) & 
                      (stats_lenti['cell']=='293T') & ~((stats_lenti['biorep']<4) & (stats_lenti['design']==3))]
marker_baseline = np.mean(data_lenti[(data_lenti['construct']=='UT') & (data_lenti['cell_type']=='293T')].groupby(['exp','biorep'])['output'].apply(sp.stats.gmean))

designs_stats_plots(axes, plot_df, marker_baseline, ylim_mean=(3e1,7e2), ylim_slope=(0,1))
axes[0].minorticks_on()
axes[0].xaxis.set_tick_params(which='minor', bottom=False)

fig.savefig(rd.outfile(output_path))

In [ ]:
plot_df = quantiles_lenti[(quantiles_lenti['group'].isin(['controller','base'])) & (quantiles_lenti['dox']==1000) & 
                             (quantiles_lenti['virus_dilution']==1) & (quantiles_lenti['cell']=='293T') &
                             (((quantiles_lenti['biorep']==1) & (quantiles_lenti['design'].isin([1,2]))) | 
                              ((quantiles_lenti['biorep']==5) & (quantiles_lenti['design'].isin([0,3]))))]
designs_list = [[0,1], [2], [3]]
marker_baseline = np.mean(data_lenti[(data_lenti['construct']=='UT') & (data_lenti['cell_type']=='293T')].groupby(['exp','biorep'])['output'].apply(sp.stats.gmean))

# Set axes limits for joint plots
xlim = (2e3,6e4)
ylim = (1e1,5e3)

subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.35, top=0.35, right=0.05)
axes = subfig.subplots(1,2,  gridspec_kw=dict(width_ratios=(1,0.4), hspace=0.88))

# line plot
ax = axes[0]
d = plot_df[plot_df['design'].isin(designs_list[0])].copy()
sns.lineplot(data=d, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
            legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax, markersize=4, markeredgewidth=0.5,
            estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', ylim=ylim, xlim=xlim)
sns.despine(ax=ax)
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.yaxis.set_tick_params(which='minor', left=False)
ax.set_title('Design 1', color=base.colors['teal'])

# histogram
ax = axes[1]
sns.kdeplot(data=d, y='output', hue='construct', palette=designs_palette, 
            legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D2']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.3, bottom=0.35, top=0.35, right=0.1)
axes = subfig.subplots(1,2,  gridspec_kw=dict(width_ratios=(1,0.4), hspace=0.88))

# line plot
ax = axes[0]
d = plot_df[plot_df['design'].isin(designs_list[1])].copy()
sns.lineplot(data=d, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
            legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax, markersize=4, markeredgewidth=0.5,
            estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', ylim=ylim, xlim=xlim, ylabel='')
sns.despine(ax=ax)
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.yaxis.set_tick_params(which='minor', left=False)
ax.set_title('Design 2', color=base.colors['orange'])

# histogram
ax = axes[1]
sns.kdeplot(data=d, y='output', hue='construct', palette=designs_palette, 
            legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D3']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.25, bottom=0.35, top=0.35, right=0.05)
axes = subfig.subplots(1,2,  gridspec_kw=dict(width_ratios=(1,0.4), hspace=0.88))

# line plot
ax = axes[0]
d = plot_df[plot_df['design'].isin(designs_list[2])].copy()
sns.lineplot(data=d, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
            legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax, markersize=4, markeredgewidth=0.5,
            estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', ylim=ylim, xlim=xlim, ylabel='')
sns.despine(ax=ax)
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.yaxis.set_tick_params(which='minor', left=False)
ax.set_title('Design 3', color=base.colors['red'])

# histogram
ax = axes[1]
sns.kdeplot(data=d, y='output', hue='construct', palette=designs_palette, 
            legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_designs.pdf'))